In [40]:
import pandas as pd
import numpy as np
from google.cloud import translate_v2 as translate

In [4]:
# Datasets built using chrome extension
# Instant Data Scraper

dsa = pd.read_csv('centris (1).csv')
dsb = pd.read_csv('centris (2).csv')

In [8]:
df = pd.concat([dsa, dsb], ignore_index=True, sort=False)

In [10]:
df.columns

Index(['pager-current', 'broker-info__broker-title',
       'broker-info__broker-title 2', 'btn', 'btn 2', 'col', 'pt-1',
       'text-nowrap', 'print href', 'btn 5', 'btn 6', 'btn 7', 'btn href',
       'btn 8', 'btn 10', 'btn 11', 'col-lg-3', 'col-lg-3 2', 'col-lg-3 3',
       'col-lg-12 2', 'carac-value', 'carac-value 2', 'carac-value 3',
       'carac-value 4', 'carac-value 7', 'property-description', 'btn 12',
       'btn 13', 'btn 14', 'btn 16', 'btn 17', 'btn 18', 'btn href 3',
       'btn href 4', 'btn href 5', 'btn 19', 'btn 20', 'btn 21', 'btn 23',
       'btn 24', 'p1 19', 'btn 29', 'walkscore', 'carac-title 8',
       'carac-value 8', 'broker-info__broker-title 4'],
      dtype='object')

In [18]:
lcol = ['broker-info__broker-title','col','pt-1', 'text-nowrap','print href',
        'property-description', 'col-lg-3', 'col-lg-3 2', 'col-lg-3 3']
df_filtered = df[lcol][~df['pt-1'].isnull()]
print(df_filtered.shape)
df_filtered.head(3)

(1582, 9)


,broker-info__broker-title,col,pt-1,text-nowrap,print href,property-description,col-lg-3,col-lg-3 2,col-lg-3 3
2,Rémi Tremblay courtier immobilier inc.,Cottage for sale,"6213, Chemin du Lac-Clair Nord, Rawdon","$1,149,000",https://www.centris.ca/en/print/property/24523790,Emplacement de premier choix! Superbe propriét...,20 rooms,5 bedrooms (3 in basement),3 bathrooms
3,Youthana Radisson Khuy,Cottage for sale,"3200, Chemin Plante, Saint-Alexis-des-Monts","$598,000",https://www.centris.ca/en/print/property/11305349,Chalet locatif court terme avec permis sur un ...,11 rooms,3 bedrooms,3 bathrooms
4,Nadine Vital,Cottage for sale or for rent,"1780, Impasse des Rapides, Amherst","$349,000",https://www.centris.ca/en/print/property/14092106,You'll be able to relax and disconnect from ev...,5 rooms,2 bedrooms (1 in basement),1 bathroom


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_text(text):
    if pd.notnull(text):
        tokens = tokenizer(text, return_tensors="pt", padding=True,
                           truncation=True)
        translation = model.generate(**tokens)
        return tokenizer.decode(translation[0], skip_special_tokens=True)
    return text

df_filtered['property-description-en'] = df_filtered['property-description'].apply(translate_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [47]:
df_filtered['Details'] = (
    df_filtered['col'].astype(str) + ': ' +
    df_filtered['pt-1'].astype(str) + ' for ' +
    df_filtered['text-nowrap'].astype(str) + ' with ' +
    df_filtered['col-lg-3'].astype(str) + ', ' +
    df_filtered['col-lg-3 2'].astype(str) + ', and ' +
    df_filtered['col-lg-3 3'].astype(str) + ' by ' +
    df_filtered['broker-info__broker-title'].astype(str) + '. ' +
    df_filtered['property-description-en'].astype(str)
)
print(df_filtered['Details'].iloc[0])

Cottage for sale: 6213, Chemin du Lac-Clair Nord, Rawdon for $1,149,000 with 20 rooms, 5 bedrooms (3 in basement), and 3 bathrooms by Rémi Tremblay courtier immobilier inc.. Great location! Superb property located on a wooded land of mature pines on the edge of Lake Clair in Lanaudière. Perfect west orientation to admire sunsets. Private sandy beach. Large land of 57,000 square feet with an exceptional frontage of 300 feet on the lake. This location is simply unique! Built in 2016, this one-storey residence offers 5 bedrooms, 3 complete bathrooms. All in all modern comfort in an enchanting setting. The large detached double garage with oversized door and the built-in propane gas generator completes this property.


In [48]:
df_filtered = df_filtered[['Details']]

In [49]:
df_filtered.to_csv('realestate_mtl_waterfront.csv')